# See if we can close a heat budget in the new runs

In [5]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.path as mpath
import matplotlib.colors as col
from netCDF4 import Dataset

import xarray as xr
import xesmf as xe

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

figdir = '/g/data/e14/pc5520/figures/basal_melt_param/'

In [6]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/38311/status,
Dashboard: /proxy/38311/status,Workers: 6
Total threads: 24,Total memory: 128.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44675,Workers: 6
Dashboard: /proxy/38311/status,Total threads: 24
Started: Just now,Total memory: 128.00 GiB
Comm: tcp://127.0.0.1:40019,Total threads: 4
Dashboard: /proxy/37597/status,Memory: 21.33 GiB
Nanny: tcp://127.0.0.1:35665,


In [7]:
#master_session = cc.database.create_session()
master_session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
control = '01deg_jra55v13_ryf9091'

session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
#session_name = '/g/data/v45/akm157/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

# temperature of meltwater using Gade line:
basal = 'accessom2-GPC015'
iceberg = 'accessom2-GPC018'


In [8]:
# build database for these experiments:
session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

dir_list = ['/g/data/e14/pc5520/access-om2/archive/01deg_jra55v13_ryf9091_rerun_for_easterlies/accessom2-GPC013/',
           '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/accessom2-GPC015/',
            '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC016/accessom2-GPC016/',
           '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/accessom2-GPC018/']
cc.database.build_index(dir_list,basal_melt_session)

Indexing experiment: accessom2-GPC013
Indexing experiment: accessom2-GPC015
Indexing experiment: accessom2-GPC016
Indexing experiment: accessom2-GPC018


0

In [10]:
session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/control.db'
master_session = cc.database.create_session(session_name)
control = '01deg_jra55v13_ryf9091'
dir_list = ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/']
cc.database.build_index(dir_list,master_session)

Indexing experiment: 01deg_jra55v13_ryf9091


100%|██████████| 399/399 [01:50<00:00,  3.62it/s]


13399

## First test for control:
This closes nicely.

In [18]:
# Make sure these match the available restarts:
start_time='2150-01-01'
end_time='2150-12-31'

# surface heat flux:
# Unfortunately net_sfc_heating diagnostic is incorrect/missing for these runs:
#net_sfc_heating = cc.querying.getvar(control, 'net_sfc_heating', master_session, start_time=start_time, end_time=end_time)
#net_sfc_heating = net_sfc_heating.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)
sfc_hflux_from_runoff = cc.querying.getvar(control, 'sfc_hflux_from_runoff', master_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
sfc_hflux_coupler     = cc.querying.getvar(control, 'sfc_hflux_coupler', master_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
sfc_hflux_pme         = cc.querying.getvar(control, 'sfc_hflux_pme', master_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
frazil_3d_int_z       = cc.querying.getvar(control, 'frazil_3d_int_z', master_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')

net_sfc_heating = sfc_hflux_from_runoff + sfc_hflux_coupler + sfc_hflux_pme + frazil_3d_int_z
net_sfc_heating = net_sfc_heating.sel(time=slice(start_time,end_time))

sfc_hflux_from_runoff = sfc_hflux_from_runoff.sel(time=slice(start_time,end_time))
sfc_hflux_coupler = sfc_hflux_coupler.sel(time=slice(start_time,end_time))
sfc_hflux_pme = sfc_hflux_pme.sel(time=slice(start_time,end_time))
frazil_3d_int_z = frazil_3d_int_z.sel(time=slice(start_time,end_time))

area = cc.querying.getvar(control, 'area_t', master_session, n=1)

# weight by days in month:
n_years = int(len(net_sfc_heating.time)/12)
sec_in_month = xr.DataArray(np.tile([31,28,31,30,31,30,31,31,30,31,30,31],n_years),coords=dict(time=net_sfc_heating.time.values),dims=['time'])*24*60*60

# This will now be in J integrated over the year.
net_sfc_heating = ((net_sfc_heating*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_from_runoff = ((sfc_hflux_from_runoff*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_coupler = ((sfc_hflux_coupler*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_pme = ((sfc_hflux_pme*sec_in_month).sum('time')*area).sum().compute()
frazil_3d_int_z = ((frazil_3d_int_z*sec_in_month).sum('time')*area).sum().compute()

#net_sfc_heating.plot(vmin=-1e17,vmax=1e17,cmap=cmocean.balance)

print('Net surface heat input over this period =',net_sfc_heating.values/1e20,'x 10^20 J')
print('sfc_hflux_from_runoff =',sfc_hflux_from_runoff.values/1e20,'x 10^20 J')
print('sfc_hflux_coupler =',sfc_hflux_coupler.values/1e20,'x 10^20 J')
print('sfc_hflux_pme =',sfc_hflux_pme.values/1e20,'x 10^20 J')
print('frazil_3d_int_z =',frazil_3d_int_z.values/1e20,'x 10^20 J')

Exception during reset or similar
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 683, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 22503362770752 and this is thread id 22498846435072.
Exception closing connection <sqlite3.Connection object at 0x1476ab3f3b70>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-pa

Net surface heat input over this period = -7.299525009243143 x 10^20 J
sfc_hflux_from_runoff = 29.65062711516774 x 10^20 J
sfc_hflux_coupler = 10.265273414760111 x 10^20 J
sfc_hflux_pme = -56.80931235393079 x 10^20 J
frazil_3d_int_z = 9.593886964436141 x 10^20 J


In [24]:
# heat content change:
# We need to calculate this from snapshots from the restarts.

temp_dummy = cc.querying.getvar(control, 'temp', master_session, n=1,frequency='1 monthly')

# note I had to collate these restarts first:
filepath = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/restart995/ocean/ocean_temp_salt.res.nc'
restart1t = xr.open_dataset(filepath)
# only 1 time snapshot:
temp_init = restart1t.temp[0,...]
# fix coords:
temp_init.coords['zaxis_1'] = temp_dummy['st_ocean'].values
temp_init.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
temp_init.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
temp_init = temp_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
temp_init = temp_init.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/restart995/ocean/ocean_thickness.res.nc'
restart1z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_init = restart1z.rho_dzt[0,...]
# fix coords:
dzt_init.coords['zaxis_1'] = temp_dummy['st_ocean'].values
dzt_init.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
dzt_init.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_init = dzt_init.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/restart1002/ocean/ocean_temp_salt.res.nc'
restart2t = xr.open_dataset(filepath)
# only 1 time snapshot:
temp_end = restart2t.temp[0,...]
# fix coords:
temp_end.coords['zaxis_1'] = temp_dummy['st_ocean'].values
temp_end.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
temp_end.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
temp_end = temp_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
temp_end = temp_end.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/restart1002/ocean/ocean_thickness.res.nc'
restart2z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_end = restart2z.rho_dzt[0,...]
# fix coords:
dzt_end.coords['zaxis_1'] = temp_dummy['st_ocean'].values
dzt_end.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
dzt_end.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
dzt_end = dzt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_end = dzt_end.drop_vars('Time')

FileNotFoundError: [Errno 2] No such file or directory: b'/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/restart995/ocean/ocean_temp_salt.res.nc'

In [ ]:
# compute heat content change:

# J / kg / degC
c_p = 3992.1

area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

# there is a factor of rho_0 in dzt here:
heat_content_change = (c_p*((temp_end*dzt_end).sum('st_ocean') - (temp_init*dzt_init).sum('st_ocean'))*area).sum()

print('Net heat content change over this period =',heat_content_change.values/1e20,'x 10^20 J')
print('Net surface heat input over this period =',net_sfc_heating.values/1e20,'x 10^20 J')
print('Heat budget residual =',(heat_content_change.values-net_sfc_heating.values)/1e20,'x 10^20 J')

## Now repeat for basal run:

In [ ]:
# Make sure these match the available restarts:
start_time='2150-01-01'
end_time='2150-12-31'

# surface heat flux:
# Unfortunately net_sfc_heating diagnostic is incorrect/missing for these runs:
#net_sfc_heating = cc.querying.getvar(control, 'net_sfc_heating', master_session, start_time=start_time, end_time=end_time)
#net_sfc_heating = net_sfc_heating.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)

sfc_hflux_from_runoff = cc.querying.getvar(basal, 'sfc_hflux_from_runoff', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
sfc_hflux_coupler     = cc.querying.getvar(basal, 'sfc_hflux_coupler', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
sfc_hflux_pme         = cc.querying.getvar(basal, 'sfc_hflux_pme', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
frazil_3d_int_z       = cc.querying.getvar(basal, 'frazil_3d_int_z', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')

net_sfc_heating = sfc_hflux_from_runoff + sfc_hflux_coupler + sfc_hflux_pme + frazil_3d_int_z
net_sfc_heating = net_sfc_heating.sel(time=slice(start_time,end_time))

sfc_hflux_from_runoff = sfc_hflux_from_runoff.sel(time=slice(start_time,end_time))
sfc_hflux_coupler = sfc_hflux_coupler.sel(time=slice(start_time,end_time))
sfc_hflux_pme = sfc_hflux_pme.sel(time=slice(start_time,end_time))
frazil_3d_int_z = frazil_3d_int_z.sel(time=slice(start_time,end_time))

area = cc.querying.getvar(control, 'area_t', master_session, n=1)

# weight by days in month:
n_years = int(len(net_sfc_heating.time)/12)
sec_in_month = xr.DataArray(np.tile([31,28,31,30,31,30,31,31,30,31,30,31],n_years),coords=dict(time=net_sfc_heating.time.values),dims=['time'])*24*60*60

# This will now be in J integrated over the year.
net_sfc_heating = ((net_sfc_heating*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_from_runoff = ((sfc_hflux_from_runoff*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_coupler = ((sfc_hflux_coupler*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_pme = ((sfc_hflux_pme*sec_in_month).sum('time')*area).sum().compute()
frazil_3d_int_z = ((frazil_3d_int_z*sec_in_month).sum('time')*area).sum().compute()

#net_sfc_heating.plot(vmin=-1e17,vmax=1e17,cmap=cmocean.balance)

print('Net surface heat input over this period =',net_sfc_heating.values/1e20,'x 10^20 J')
print('sfc_hflux_from_runoff =',sfc_hflux_from_runoff.values/1e20,'x 10^20 J')
print('sfc_hflux_coupler =',sfc_hflux_coupler.values/1e20,'x 10^20 J')
print('sfc_hflux_pme =',sfc_hflux_pme.values/1e20,'x 10^20 J')
print('frazil_3d_int_z =',frazil_3d_int_z.values/1e20,'x 10^20 J')

In [ ]:
# temp_basalmix

basal_heat_flux = cc.querying.getvar(basal, 'temp_basalmix', basal_melt_session, start_time=start_time, end_time=end_time,frequency='1 monthly')
basal_heat_flux = basal_heat_flux.sel(time=slice(start_time,end_time))

area = cc.querying.getvar(control, 'area_t', master_session, n=1)

# weight by days in month:
n_years = int(len(basal_heat_flux.time)/12)
sec_in_month = xr.DataArray(np.tile([31,28,31,30,31,30,31,31,30,31,30,31],n_years),coords=dict(time=basal_heat_flux.time.values),dims=['time'])*24*60*60

# This will now be in J integrated over the year.
basal_heat_flux = ((basal_heat_flux*sec_in_month).sum('time').sum('st_ocean')*area).sum().compute()

print('Net basal heat flux over this period =',basal_heat_flux.values/1e20,'x 10^20 J')

In [ ]:
# heat content change:
# We need to calculate this from snapshots from the restarts.

temp_dummy = cc.querying.getvar(control, 'temp', master_session, n=1,frequency='1 monthly')

# Can use control restart for first one at start of basal simulation:
filepath = '/scratch/v45/akm157/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
restart1t = xr.open_dataset(filepath)
# only 1 time snapshot:
temp_init = restart1t.temp[0,...]
# fix coords:
temp_init.coords['zaxis_1'] = temp_dummy['st_ocean'].values
temp_init.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
temp_init.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
temp_init = temp_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
temp_init = temp_init.drop_vars('Time')

filepath = '/scratch/v45/akm157/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
restart1z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_init = restart1z.rho_dzt[0,...]
# fix coords:
dzt_init.coords['zaxis_1'] = temp_dummy['st_ocean'].values
dzt_init.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
dzt_init.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_init = dzt_init.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/'+basal+'/restarts/restart1002/ocean/ocean_temp_salt.res.nc'
restart2t = xr.open_dataset(filepath)
# only 1 time snapshot:
temp_end = restart2t.temp[0,...]
# fix coords:
temp_end.coords['zaxis_1'] = temp_dummy['st_ocean'].values
temp_end.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
temp_end.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
temp_end = temp_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
temp_end = temp_end.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/'+basal+'/restarts/restart1002/ocean/ocean_thickness.res.nc'
restart2z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_end = restart2z.rho_dzt[0,...]
# fix coords:
dzt_end.coords['zaxis_1'] = temp_dummy['st_ocean'].values
dzt_end.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
dzt_end.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
dzt_end = dzt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_end = dzt_end.drop_vars('Time')

In [ ]:
# compute heat content change:

# J / kg / degC
c_p = 3992.1

area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

# there is a factor of rho_0 in dzt here:
heat_content_change = (c_p*((temp_end*dzt_end).sum('st_ocean') - (temp_init*dzt_init).sum('st_ocean'))*area).sum()

print('Net heat content change over this period =',heat_content_change.values/1e20,'x 10^20 J')
print('Net surface heat input plus basal over this period =',(net_sfc_heating.values + basal_heat_flux.values)/1e20,'x 10^20 J')
print('Heat budget residual =',(heat_content_change.values - net_sfc_heating.values - basal_heat_flux.values)/1e20,'x 10^20 J')

# For ICB run

In [20]:
# Make sure these match the available restarts:
start_time='2150-01-01'
end_time='2150-12-31'

# surface heat flux:
# Unfortunately net_sfc_heating diagnostic is incorrect/missing for these runs:
#net_sfc_heating = cc.querying.getvar(control, 'net_sfc_heating', master_session, start_time=start_time, end_time=end_time)
#net_sfc_heating = net_sfc_heating.sel(time=slice(start_time,end_time)).sel(yt_ocean=lat_slice).sel(xt_ocean=lon_slice)

sfc_hflux_from_runoff = cc.querying.getvar(iceberg, 'sfc_hflux_from_runoff', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
sfc_hflux_coupler     = cc.querying.getvar(iceberg, 'sfc_hflux_coupler', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
sfc_hflux_pme         = cc.querying.getvar(iceberg, 'sfc_hflux_pme', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')
frazil_3d_int_z       = cc.querying.getvar(iceberg, 'frazil_3d_int_z', basal_melt_session, start_time=start_time, end_time=end_time,ncfile='ocean_month.nc')

net_sfc_heating = sfc_hflux_from_runoff + sfc_hflux_coupler + sfc_hflux_pme + frazil_3d_int_z
net_sfc_heating = net_sfc_heating.sel(time=slice(start_time,end_time))

sfc_hflux_from_runoff = sfc_hflux_from_runoff.sel(time=slice(start_time,end_time))
sfc_hflux_coupler = sfc_hflux_coupler.sel(time=slice(start_time,end_time))
sfc_hflux_pme = sfc_hflux_pme.sel(time=slice(start_time,end_time))
frazil_3d_int_z = frazil_3d_int_z.sel(time=slice(start_time,end_time))

area = cc.querying.getvar(control, 'area_t', master_session, n=1)

# weight by days in month:
n_years = int(len(net_sfc_heating.time)/12)
sec_in_month = xr.DataArray(np.tile([31,28,31,30,31,30,31,31,30,31,30,31],n_years),coords=dict(time=net_sfc_heating.time.values),dims=['time'])*24*60*60

# This will now be in J integrated over the year.
net_sfc_heating = ((net_sfc_heating*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_from_runoff = ((sfc_hflux_from_runoff*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_coupler = ((sfc_hflux_coupler*sec_in_month).sum('time')*area).sum().compute()
sfc_hflux_pme = ((sfc_hflux_pme*sec_in_month).sum('time')*area).sum().compute()
frazil_3d_int_z = ((frazil_3d_int_z*sec_in_month).sum('time')*area).sum().compute()

#net_sfc_heating.plot(vmin=-1e17,vmax=1e17,cmap=cmocean.balance)

print('Net surface heat input over this period =',net_sfc_heating.values/1e20,'x 10^20 J')
print('sfc_hflux_from_runoff =',sfc_hflux_from_runoff.values/1e20,'x 10^20 J')
print('sfc_hflux_coupler =',sfc_hflux_coupler.values/1e20,'x 10^20 J')
print('sfc_hflux_pme =',sfc_hflux_pme.values/1e20,'x 10^20 J')
print('frazil_3d_int_z =',frazil_3d_int_z.values/1e20,'x 10^20 J')

Net surface heat input over this period = -2.186040236772027 x 10^20 J
sfc_hflux_from_runoff = 29.63164671740925 x 10^20 J
sfc_hflux_coupler = 13.328913120229075 x 10^20 J
sfc_hflux_pme = -56.73712781786658 x 10^20 J
frazil_3d_int_z = 11.590527866377233 x 10^20 J


In [21]:
# temp_basalmix

basal_heat_flux = cc.querying.getvar(iceberg, 'temp_basalmix', basal_melt_session, start_time=start_time, end_time=end_time,frequency='1 monthly')
basal_heat_flux = basal_heat_flux.sel(time=slice(start_time,end_time))

area = cc.querying.getvar(control, 'area_t', master_session, n=1)

# weight by days in month:
n_years = int(len(basal_heat_flux.time)/12)
sec_in_month = xr.DataArray(np.tile([31,28,31,30,31,30,31,31,30,31,30,31],n_years),coords=dict(time=basal_heat_flux.time.values),dims=['time'])*24*60*60

# This will now be in J integrated over the year.
basal_heat_flux = ((basal_heat_flux*sec_in_month).sum('time').sum('st_ocean')*area).sum().compute()

print('Net basal heat flux over this period =',basal_heat_flux.values/1e20,'x 10^20 J')

Net basal heat flux over this period = -4.847541864219881 x 10^20 J


In [22]:
# temp_icbmix

icb_heat_flux = cc.querying.getvar(iceberg, 'temp_icbmix', basal_melt_session, start_time=start_time, end_time=end_time,frequency='1 monthly')
icb_heat_flux = icb_heat_flux.sel(time=slice(start_time,end_time))

area = cc.querying.getvar(control, 'area_t', master_session, n=1)

# weight by days in month:
n_years = int(len(icb_heat_flux.time)/12)
sec_in_month = xr.DataArray(np.tile([31,28,31,30,31,30,31,31,30,31,30,31],n_years),coords=dict(time=icb_heat_flux.time.values),dims=['time'])*24*60*60

# This will now be in J integrated over the year.
icb_heat_flux = ((icb_heat_flux*sec_in_month).sum('time').sum('st_ocean')*area).sum().compute()

print('Net icb heat flux over this period =',icb_heat_flux.values/1e20,'x 10^20 J')

2023-03-07 17:06:43,121 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-03-07 17:06:45,170 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


Net icb heat flux over this period = -4.412215750860296 x 10^20 J


In [23]:
# heat content change:
# We need to calculate this from snapshots from the restarts.

temp_dummy = cc.querying.getvar(control, 'temp', master_session, n=1,frequency='1 monthly')

# Can use control restart for first one at start of basal simulation:
filepath = '/scratch/v45/akm157/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
restart1t = xr.open_dataset(filepath)
# only 1 time snapshot:
temp_init = restart1t.temp[0,...]
# fix coords:
temp_init.coords['zaxis_1'] = temp_dummy['st_ocean'].values
temp_init.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
temp_init.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
temp_init = temp_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
temp_init = temp_init.drop_vars('Time')

filepath = '/scratch/v45/akm157/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
restart1z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_init = restart1z.rho_dzt[0,...]
# fix coords:
dzt_init.coords['zaxis_1'] = temp_dummy['st_ocean'].values
dzt_init.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
dzt_init.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_init = dzt_init.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/'+iceberg+'/restarts/restart1007/ocean/ocean_temp_salt.res.nc'
restart2t = xr.open_dataset(filepath)
# only 1 time snapshot:
temp_end = restart2t.temp[0,...]
# fix coords:
temp_end.coords['zaxis_1'] = temp_dummy['st_ocean'].values
temp_end.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
temp_end.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
temp_end = temp_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
temp_end = temp_end.drop_vars('Time')

filepath = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/'+iceberg+'/restarts/restart1007/ocean/ocean_thickness.res.nc'
restart2z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_end = restart2z.rho_dzt[0,...]
# fix coords:
dzt_end.coords['zaxis_1'] = temp_dummy['st_ocean'].values
dzt_end.coords['yaxis_1'] = temp_dummy['yt_ocean'].values
dzt_end.coords['xaxis_1'] = temp_dummy['xt_ocean'].values
dzt_end = dzt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_end = dzt_end.drop_vars('Time')

FileNotFoundError: [Errno 2] No such file or directory: b'/scratch/v45/akm157/access-om2/archive/01deg_jra55v13_ryf9091/restart995/ocean/ocean_temp_salt.res.nc'

In [ ]:
# compute heat content change:

# J / kg / degC
c_p = 3992.1

area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

# there is a factor of rho_0 in dzt here:
heat_content_change = (c_p*((temp_end*dzt_end).sum('st_ocean') - (temp_init*dzt_init).sum('st_ocean'))*area).sum()

print('Net heat content change over this period =',heat_content_change.values/1e20,'x 10^20 J')
print('Net surface heat input plus basal+icb over this period =',(net_sfc_heating.values + basal_heat_flux.values+icb_heat_flux.values)/1e20,'x 10^20 J')
print('Heat budget residual =',(heat_content_change.values - net_sfc_heating.values - basal_heat_flux.values - icb_heat_flux.values)/1e20,'x 10^20 J')